In [5]:
import pandas as pd
#import stumpy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt
import scipy
import sklearn
import copy
import math
from tabulate import tabulate

plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/'
DATASET_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/k6csv/'
Baseline_DATA_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/k6csv/Baselines/'
Aggregate_DATA_PATH = '/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/k6csv/RngRemake/'

import json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
jsonName = 'nov24_24hours'
f = open(DATA_PATH+'/k6json/'+jsonName+'.json')
data = json.load(f)
#print(data['scenarios']['secondScenario']['stages'])
bubbleTargets = []
bogoTargets = []
for i in data['scenarios']['firstScenario']['stages']:
  if i['duration'] == '1m':
    bubbleTargets.append(int(i['target']))
for i in data['scenarios']['secondScenario']['stages']:
  if i['duration'] == '1m':
    bogoTargets.append(int(i['target']))

In [7]:
jsonName = 'nov24_24hours_1445'
f = open(DATA_PATH+'/k6json/'+jsonName+'.json')
data = json.load(f)
#print(data['scenarios']['secondScenario']['stages'])
bubbleTargets2 = []
bogoTargets2 = []
for i in data['scenarios']['firstScenario']['stages']:
  if i['duration'] == '1m' and i['target'] != 0:
    bubbleTargets2.append(int(i['target']))
for i in data['scenarios']['secondScenario']['stages']:
  if i['duration'] == '1m' and i['target'] != 0:
    bogoTargets2.append(int(i['target']))

In [8]:
print(sum(bubbleTargets)/len(bubbleTargets))
print(sum(bogoTargets)/len(bogoTargets))
print(sum(bubbleTargets2)/len(bubbleTargets2))
print(sum(bogoTargets2)/len(bogoTargets2))

3138.029861111111
2215.077083333333
2764.3777777777777
1748.5958333333333


In [ ]:
bins, binEdges = np.histogram(bubbleTargets,bins='fd')
bins2, binEdges2 = np.histogram(bubbleTargets2,bins=binEdges)
bins2 = bins2
obs = np.array([bins,bins2])
obsPer = [bins/len(bubbleTargets),bins2/len(bubbleTargets2)]
print(obs.T)
print(obsPer)
print(binEdges)

[[   2 1080]
 [ 160    0]
 [  87    0]
 [ 111    0]
 [  82    0]
 [  79    0]
 [  72    0]
 [ 130    0]
 [ 175    0]
 [ 248    1]
 [ 267   21]
 [  27  338]]
[array([0.00138889, 0.11111111, 0.06041667, 0.07708333, 0.05694444,
       0.05486111, 0.05      , 0.09027778, 0.12152778, 0.17222222,
       0.18541667, 0.01875   ]), array([7.50000000e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 6.94444444e-04, 1.45833333e-02, 2.34722222e-01])]
[2500.         2591.66666667 2683.33333333 2775.         2866.66666667
 2958.33333333 3050.         3141.66666667 3233.33333333 3325.
 3416.66666667 3508.33333333 3600.        ]


In [ ]:
bubbleBins, bubbleBinEdges = np.histogram(bubbleTargets,bins='fd')
print(np.around(bubbleBins/len(bubbleTargets),2), bubbleBinEdges)
bogoBins, bogoBinEdges = np.histogram(bogoTargets,bins=len(bubbleBins))
print(np.around(bogoBins/len(bogoTargets),2), bogoBinEdges)

[0.   0.11 0.06 0.08 0.06 0.05 0.05 0.09 0.12 0.17 0.19 0.02] [2500.         2591.66666667 2683.33333333 2775.         2866.66666667
 2958.33333333 3050.         3141.66666667 3233.33333333 3325.
 3416.66666667 3508.33333333 3600.        ]
[0.   0.   0.   0.   0.01 0.05 0.07 0.05 0.06 0.1  0.39 0.28] [1600.  1662.5 1725.  1787.5 1850.  1912.5 1975.  2037.5 2100.  2162.5
 2225.  2287.5 2350. ]


In [ ]:
save = True
json_name = "nov24_24hours_1445"

stage1 = []
stage2 = []

stage2.append({ "duration": '5m', "target": 1800 })

stage1.append({ "duration": '5m', "target": 2800 })

newJson = {
    "scenarios": {
      "firstScenario": {
        "exec": "execute_get_bubble",
        "startRate": 0,
        "startTime": '0s',
        "timeUnit": '1m',
        "preAllocatedVUs": 500,
        "maxVUs": 1000,
        "executor": 'ramping-arrival-rate',
        "stages": stage1,
        "gracefulStop": '2m'
    },
      "secondScenario": {
        "exec": "execute_get_bogo",
        "startRate": 0,
        "startTime": '0s',
        "timeUnit": '1m',
        "preAllocatedVUs": 500,
        "maxVUs": 1000,
        "executor": 'ramping-arrival-rate',
        "stages": stage2,
        "gracefulStop": '2m'
    }
  }
}


if save:
  with open(f'/content/drive/MyDrive/Project/AzureDataset_FunctionCalls/k6json/{json_name}.json', 'w', encoding='utf-8') as f: json.dump(newJson, f, ensure_ascii=False, indent=4, default=str)